In [ ]:
import numpy as np
from numpy import shape

from datetime import date

import random
import math
import dill
import glob
import gc

# from tensorflow import keras

import matplotlib.pyplot as plt

import tensorflow as tf

from random import seed

# Import train_test_split function
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import device

# Tensorflow soll auf CPU und nicht auf der GPU laufen
device("cpu:0")

# für GPU:
# tf.device("gpu:0")

In [ ]:
%run '../../_functions/functions.py'

### Load MNIST images

In [ ]:
# # MNIST von sklearn mit 28x28 Pixel
# from sklearn.datasets import fetch_openml

# # Load data from https://www.openml.org/d/554
# X_raw, y_raw = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False)

In [ ]:
# Import data that was saved on this system

raw_data = load_file(file = "../raw_data/MNIST_images.pkl")
X_raw = raw_data[0]
y_raw = raw_data[1]

In [ ]:
# Filter on indices of '0' and '8'
indices = (y_raw == '0') | (y_raw == '8')

# Reshape into squares
X_raw = X_raw[indices, :]
X = X_raw.reshape([sum(indices), 28, 28])

y = y_raw[indices]

# Factorize into dichotomous variable
Y = pd.factorize(y)[0]

In [ ]:
# We analyse 1998 samples
np.unique(y[0:1998], return_counts=True)

### Load Persistences Landscapes

In [ ]:
pl_folderpath = "../persistence_landscapes_averages/"

In [ ]:
# Eingelesenen Dateien
print(glob.glob(pl_folderpath + "avgPL_bucket01_H0_scaledWithin*.pkl")[-1])
print(glob.glob(pl_folderpath + "avgPL_bucket01_H1_scaledWithin*.pkl")[-1])
print(glob.glob(pl_folderpath + "avgPL_bucket01_H0_scaledBetween*.pkl")[-1])
print(glob.glob(pl_folderpath + "avgPL_bucket01_H1_scaledBetween*.pkl")[-1])
print(glob.glob(pl_folderpath + "avgPL_bucket01_H0_unscaled*.pkl")[-1])
print(glob.glob(pl_folderpath + "avgPL_bucket01_H1_unscaled*.pkl")[-1])

In [ ]:
avgPL_bucket01_H0_scaledWithin = load_file(file = glob.glob(pl_folderpath + "avgPL_bucket01_H0_scaledWithin*.pkl")[-1])
avgPL_bucket01_H1_scaledWithin = load_file(file = glob.glob(pl_folderpath + "avgPL_bucket01_H1_scaledWithin*.pkl")[-1])

avgPL_bucket01_H0_scaledBetween = load_file(file = glob.glob(pl_folderpath + "avgPL_bucket01_H0_scaledBetween*.pkl")[-1])
avgPL_bucket01_H1_scaledBetween = load_file(file = glob.glob(pl_folderpath + "avgPL_bucket01_H1_scaledBetween*.pkl")[-1])

avgPL_bucket01_H0_unscaled = load_file(file = glob.glob(pl_folderpath + "avgPL_bucket01_H0_unscaled*.pkl")[-1])
avgPL_bucket01_H1_unscaled = load_file(file = glob.glob(pl_folderpath + "avgPL_bucket01_H1_unscaled*.pkl")[-1])

In [ ]:
# Parameter for persistence landscapes
pl_resolution = 250
pl_num_landscapes = 10

### Train-Test-Splits Persistence Landscapes

In [ ]:
relevant_samples = len(avgPL_bucket01_H0_scaledBetween)
relevant_samples

In [ ]:
# Bucket01 scaledBetween
seed(999)
# Train-Test-Split 
X_b01_scaledBetween_H0_train, X_b01_scaledBetween_H0_test, Y_b01_scaledBetween_H0_train, Y_b01_scaledBetween_H0_test = train_test_split(avgPL_bucket01_H0_scaledBetween,
                                                                                                                                        Y[0:relevant_samples],
                                                                                                                                        test_size=0.2)

seed(999)
# Train-Test-Split 
X_b01_scaledBetween_H1_train, X_b01_scaledBetween_H1_test, Y_b01_scaledBetween_H1_train, Y_b01_scaledBetween_H1_test = train_test_split(avgPL_bucket01_H1_scaledBetween,
                                                                                                                                        Y[0:relevant_samples],
                                                                                                                                        test_size=0.2)

# Bucket01 scaledWithin
seed(999)
# Train-Test-Split 
X_b01_scaledWithin_H0_train, X_b01_scaledWithin_H0_test, Y_b01_scaledWithin_H0_train, Y_b01_scaledWithin_H0_test = train_test_split(avgPL_bucket01_H0_scaledWithin,
                                                                                                                                    Y[0:relevant_samples],
                                                                                                                                    test_size=0.2)
seed(999)
# Train-Test-Split 
X_b01_scaledWithin_H1_train, X_b01_scaledWithin_H1_test, Y_b01_scaledWithin_H1_train, Y_b01_scaledWithin_H1_test = train_test_split(avgPL_bucket01_H1_scaledWithin,
                                                                                                                                    Y[0:relevant_samples],
                                                                                                                                    test_size=0.2)

# Bucket01 Unscaled
seed(999)
# Train-Test-Split 
X_b01_unscaled_H0_train, X_b01_unscaled_H0_test, Y_b01_unscaled_H0_train, Y_b01_unscaled_H0_test = train_test_split(avgPL_bucket01_H0_unscaled,
                                                                                                                    Y[0:relevant_samples],
                                                                                                                    test_size=0.2)
seed(999)
# Train-Test-Split 
X_b01_unscaled_H1_train, X_b01_unscaled_H1_test, Y_b01_unscaled_H1_train, Y_b01_unscaled_H1_test = train_test_split(avgPL_bucket01_H1_unscaled,
                                                                                                                    Y[0:relevant_samples],
                                                                                                                    test_size=0.2)

# Machine Learning Classification

## Support Vector Machine

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

In [ ]:
folderpath_results = "../results/"

In [ ]:
# Train-test split with data in vector format
image_vector_size = 28*28

X_raw_train, X_raw_test, y_raw_train, y_raw_test = train_test_split(X[0:len(avgPL_bucket01_H1_scaledBetween)].reshape(len(avgPL_bucket01_H1_scaledBetween), image_vector_size),
                                                                    np.asarray(y[0:len(avgPL_bucket01_H1_scaledBetween)]),
                                                                    test_size = 0.3,
                                                                    random_state = 2)

##### SVM (raw data)

In [ ]:
# Linear - Rawdata
clf = make_pipeline(StandardScaler(), 
                    SVC(kernel='linear')).fit(X_raw_train, y_raw_train.ravel())

print("Accuracy SVM on raw data: " + str(clf.score(X_raw_test, y_raw_test))[:10])

evaluation(y_test = y_raw_test,
           y_pred = clf.predict(X_raw_test),
           filename_csv = folderpath_results + "svm_linear_rawdata.csv")

In [ ]:
# RBF - Rawdata
clf = make_pipeline(StandardScaler(), 
                    SVC(kernel='rbf')).fit(X_raw_train, y_raw_train.ravel())

print("Accuracy SVM on raw data: " + str(clf.score(X_raw_test, y_raw_test))[:10])

evaluation(y_test = y_raw_test,
           y_pred = clf.predict(X_raw_test),
           filename_csv = folderpath_results + "svm_rbf_rawdata.csv")

##### SVM (Persistent Landscapes - scaledWithin)

In [ ]:
# Linear - H0 - scaledWithin
clf_pl_h0 = make_pipeline(StandardScaler(), 
                       SVC(kernel='linear')).fit(X_b01_scaledWithin_H0_train, Y_b01_scaledWithin_H0_train)

print("Accuracy SVM on peristence landscapes (scaled within sample): " + str(clf_pl_h0.score(X_b01_scaledWithin_H0_test, Y_b01_scaledWithin_H0_test))[:10])

evaluation(y_test = Y_b01_scaledWithin_H0_test,
           y_pred = clf_pl_h0.predict(X_b01_scaledWithin_H0_test),
           filename_csv = folderpath_results + "svm_linear_H0_scaledWithin.csv")

In [ ]:
# Linear - H1 - scaledWithin
clf_pl = make_pipeline(StandardScaler(), 
                       SVC(kernel='linear')).fit(X_b01_scaledWithin_H1_train, Y_b01_scaledWithin_H1_train)

print("Accuracy SVM on peristence landscapes (scaled within sample): " + str(clf_pl.score(X_b01_scaledWithin_H1_test, Y_b01_scaledWithin_H1_test))[:10])

evaluation(y_test = Y_b01_scaledWithin_H1_test,
           y_pred = clf_pl.predict(X_b01_scaledWithin_H1_test),
           filename_csv = folderpath_results + "svm_linear_H1_scaledWithin.csv")

In [ ]:
# RBF - H0 - scaledWithin
clf_pl_h0 = make_pipeline(StandardScaler(), 
                       SVC(kernel='rbf')).fit(X_b01_scaledWithin_H0_train, Y_b01_scaledWithin_H0_train)

print("Accuracy SVM on peristence landscapes (scaled within sample): " + str(clf_pl_h0.score(X_b01_scaledWithin_H0_test, Y_b01_scaledWithin_H0_test))[:10])

evaluation(y_test = Y_b01_scaledWithin_H0_test,
           y_pred = clf_pl_h0.predict(X_b01_scaledWithin_H0_test),
           filename_csv = folderpath_results + "svm_rbf_H0_scaledWithin.csv")

In [ ]:
# RBF - H1 - scaledWithin
clf_pl_rbf = make_pipeline(StandardScaler(), 
                       SVC(kernel='rbf')).fit(X_b01_scaledWithin_H1_train, Y_b01_scaledWithin_H1_train)

print("Accuracy SVM on peristence landscapes (scaled within sample): " + str(clf_pl_rbf.score(X_b01_scaledWithin_H1_test, Y_b01_scaledWithin_H1_test))[:10])

evaluation(y_test = Y_b01_scaledWithin_H1_test,
           y_pred = clf_pl_rbf.predict(X_b01_scaledWithin_H1_test),
           filename_csv = folderpath_results + "svm_rbf_H1_scaledWithin.csv")

##### SVM (Persistent Landscapes - scaledBetween)

In [ ]:
# Linear - H0 - scaledBetween
clf_pl = make_pipeline(StandardScaler(), 
                    SVC(kernel='linear')).fit(X_b01_scaledBetween_H0_train, Y_b01_scaledBetween_H0_train)

print("Accuracy SVM : " + str(clf_pl.score(X_b01_scaledBetween_H0_test, Y_b01_scaledBetween_H0_test))[:10])

evaluation(y_test = Y_b01_scaledBetween_H0_test,
           y_pred = clf_pl.predict(X_b01_scaledBetween_H0_test),
           filename_csv = folderpath_results + "svm_linear_H0_scaledBetween.csv")

In [ ]:
# Linear - H1 - scaledBetween
clf_pl = make_pipeline(StandardScaler(), 
                    SVC(kernel='linear')).fit(X_b01_scaledBetween_H1_train, Y_b01_scaledBetween_H1_train)

print("Accuracy SVM : " + str(clf_pl.score(X_b01_scaledBetween_H1_test, Y_b01_scaledBetween_H1_test))[:10])

evaluation(y_test = Y_b01_scaledBetween_H1_test,
           y_pred = clf_pl.predict(X_b01_scaledBetween_H1_test),
           filename_csv = folderpath_results + "svm_linear_H1_scaledBetween.csv")

In [ ]:
# RBF - H0 - scaledBetween
clf_pl = make_pipeline(StandardScaler(), 
                    SVC(kernel='rbf')).fit(X_b01_scaledBetween_H0_train, Y_b01_scaledBetween_H0_train)

print("Accuracy SVM : " + str(clf_pl.score(X_b01_scaledBetween_H0_test, Y_b01_scaledBetween_H0_test))[:10])

evaluation(y_test = Y_b01_scaledBetween_H0_test,
           y_pred = clf_pl.predict(X_b01_scaledBetween_H0_test),
           filename_csv = folderpath_results + "svm_rbf_H0_scaledBetween.csv")

In [ ]:
# RBF - H1 - scaledBetween
clf_pl_rbf = make_pipeline(StandardScaler(), 
                    SVC(kernel='rbf')).fit(X_b01_scaledBetween_H1_train, Y_b01_scaledBetween_H1_train)

print("Accuracy SVM : " + str(clf_pl_rbf.score(X_b01_scaledBetween_H1_test, Y_b01_scaledBetween_H1_test))[:10])

evaluation(y_test = Y_b01_scaledBetween_H1_test,
           y_pred = clf_pl_rbf.predict(X_b01_scaledBetween_H1_test),
           filename_csv = folderpath_results + "svm_rbf_H1_scaledBetween.csv")

##### SVM (Persistent Landscapes - unscaled)

In [ ]:
# Linear - H0 - unscaled
clf_pl = make_pipeline(StandardScaler(), 
                    SVC(kernel='linear')).fit(X_b01_unscaled_H0_train, Y_b01_unscaled_H0_train)

print("Accuracy SVM : " + str(clf_pl.score(X_b01_unscaled_H0_test, Y_b01_unscaled_H0_test))[:10])

evaluation(y_test = Y_b01_unscaled_H0_test,
           y_pred = clf_pl.predict(X_b01_unscaled_H0_test),
           filename_csv = folderpath_results + "svm_linear_H0_unscaled.csv")

In [ ]:
# Linear - H1 - unscaled
clf_pl = make_pipeline(StandardScaler(), 
                    SVC(kernel='linear')).fit(X_b01_unscaled_H1_train, Y_b01_unscaled_H1_train)

print("Accuracy SVM : " + str(clf_pl.score(X_b01_unscaled_H1_test, Y_b01_unscaled_H1_test))[:10])

evaluation(y_test = Y_b01_unscaled_H1_test,
           y_pred = clf_pl.predict(X_b01_unscaled_H1_test),
           filename_csv = folderpath_results + "svm_linear_H1_unscaled.csv")

In [ ]:
# RBF - H0 - unscaled
clf_pl = make_pipeline(StandardScaler(), 
                    SVC(kernel='rbf')).fit(X_b01_unscaled_H0_train, Y_b01_unscaled_H0_train)

print("Accuracy SVM : " + str(clf_pl.score(X_b01_unscaled_H0_test, Y_b01_unscaled_H0_test))[:10])

evaluation(y_test = Y_b01_unscaled_H0_test,
           y_pred = clf_pl.predict(X_b01_unscaled_H0_test),
           filename_csv = folderpath_results + "svm_rbf_H0_unscaled.csv")

In [ ]:
# RBF - H1 - unscaled
clf_pl_rbf = make_pipeline(StandardScaler(), 
                    SVC(kernel='rbf')).fit(X_b01_unscaled_H1_train, Y_b01_unscaled_H1_train)

print("Accuracy SVM : " + str(clf_pl_rbf.score(X_b01_unscaled_H1_test, Y_b01_unscaled_H1_test))[:10])

evaluation(y_test = Y_b01_unscaled_H1_test,
           y_pred = clf_pl_rbf.predict(X_b01_unscaled_H1_test),
           filename_csv = folderpath_results + "svm_rbf_H1_unscaled.csv")